In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import collections
import matplotlib as plt

In [2]:
profiles = pd.read_csv('profiles.csv')
scaler = StandardScaler()
print(profiles.head())
print(profiles.info())

   age       body_type               diet    drinks      drugs  \
0   22  a little extra  strictly anything  socially      never   
1   35         average       mostly other     often  sometimes   
2   38            thin           anything  socially        NaN   
3   23            thin         vegetarian  socially        NaN   
4   29        athletic                NaN  socially      never   

                           education  \
0      working on college/university   
1              working on space camp   
2     graduated from masters program   
3      working on college/university   
4  graduated from college/university   

                                              essay0  \
0  about me:<br />\n<br />\ni would love to think...   
1  i am a chef: this is what that means.<br />\n1...   
2  i'm not ashamed of much, but writing public te...   
3          i work in a library and go to school. . .   
4  hey how's it going? currently vague on the pro...   

                         

In [3]:
print(profiles['education'].value_counts())

education
graduated from college/university    23959
graduated from masters program        8961
working on college/university         5712
working on masters program            1683
graduated from two-year college       1531
graduated from high school            1428
graduated from ph.d program           1272
graduated from law school             1122
working on two-year college           1074
dropped out of college/university      995
working on ph.d program                983
college/university                     801
graduated from space camp              657
dropped out of space camp              523
graduated from med school              446
working on space camp                  445
working on law school                  269
two-year college                       222
working on med school                  212
dropped out of two-year college        191
dropped out of masters program         140
masters program                        136
dropped out of ph.d program            127
d

From these counts, it is clear that OKCupid has a large user base within graduated university students. This includes a significant number of people that continued their education and graduated from a masters degree program. This suggests a possible connection between education level and usage of OKCupid

In [4]:
#Gathering all profiles 'education' details
education = profiles['education']
#print(education.values)
#edu_labs= [profiles['education'].unique() for profiles['education'] in profiles['education']]
profiles['grad_type']=[0 if i=='graduated from college/university' or i == 'graduated from masters program' else 1 for i in profiles['education']]

print(profiles['grad_type'])

0        1
1        1
2        0
3        1
4        0
        ..
59941    0
59942    1
59943    0
59944    1
59945    0
Name: grad_type, Length: 59946, dtype: int64


In [5]:
drugs = profiles['drugs']
drugs.value_counts()



drugs
never        37724
sometimes     7732
often          410
Name: count, dtype: int64

This shows that the vast majority of profiles say they do not take drugs. This may be due to the profile owners not wanting to make it public that they use recreational drugs.

In [6]:

drugs = drugs.replace({
    'never':0,
    'sometimes':1,
    'often':2,
})
drugs= drugs.fillna(0)

In [7]:
print(drugs)

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
59941    0.0
59942    1.0
59943    0.0
59944    2.0
59945    0.0
Name: drugs, Length: 59946, dtype: float64


In [8]:
smokes = profiles['smokes']
smokes.value_counts()

smokes
no                43896
sometimes          3787
when drinking      3040
yes                2231
trying to quit     1480
Name: count, dtype: int64

In [9]:
smokes = smokes.replace({
    'no':0,
    'sometimes':1,
    'when drinking':1,
    'yes':1,
    'trying to quit':1
})

In [10]:
print(smokes)

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
59941    0.0
59942    0.0
59943    0.0
59944    1.0
59945    1.0
Name: smokes, Length: 59946, dtype: float64


In [11]:
drinks = profiles['drinks']
drinks.value_counts()

drinks
socially       41780
rarely          5957
often           5164
not at all      3267
very often       471
desperately      322
Name: count, dtype: int64

This would suggest that most OkCupid users are very likely to drink if they are in a social setting, but not necessarily alone.

In [12]:
model = nn.Sequential(
    nn.Linear(41962,20981),
    nn.ReLU(),
    nn.Linear(20981,10490),
    nn.ReLU(),
    nn.Linear(10490,5245),
    nn.ReLU(),
    nn.Linear(5245,2622),
    nn.ReLU(),
    nn.Linear(2622, 1311),
    nn.ReLU(),
    nn.Linear(1311, 655),
    nn.ReLU(),
    nn.Linear(655, 1)
)

#predictions = model()

In [13]:
torch.manual_seed(13)

X= torch.tensor(smokes.values, dtype= torch.float)#.view(X.size(0), -1)
y= torch.tensor(profiles['grad_type'].values, dtype= torch.float)#.view(y.size(0), -1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state= 13)
loss= nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    predictions = model(X_train)
    MSE = loss(predictions,y_train) # compute loss
    MSE.backward() # compute gradients
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 100 == 0:
        print(f'Epoch is {epoch} loss is {MSE}')


C:\Users\Vapou\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([41962])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch is 0 loss is nan


KeyboardInterrupt: 